In [17]:
from typing import List, Optional, Any
from datetime import datetime, date, timezone, timedelta
from dataclasses import dataclass

from sshtunnel import SSHTunnelForwarder
from tqdm import tqdm
import paramiko
import pandas as pd
import psycopg2 as pg
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB
import warnings
warnings.filterwarnings(action='ignore')

In [18]:
NST = timezone(timedelta(hours=1))

In [19]:
def start_of_day(dt: datetime) -> datetime:
    return dt.replace(hour=0, minute=0, second=0, microsecond=0)

now = datetime.now(NST)

In [20]:
ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = 'ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [21]:
def run_query(qry: str, params: tuple = ()) -> List[Any]:
    try:
        with conn.cursor() as cursor:
            cursor.execute(qry, params)
            res = cursor.fetchall()
    except Exception as e:
        conn.rollback()
        raise e
    return res


In [22]:
@dataclass
class User:
    id: str
    bs_success: bool
    bs_fail_reason: Optional[str]
    cb_done: bool
    cb_success: bool

    def to_dict(self) -> dict:
        return {k: v for k, v in self.__dict__.items() if v is not None}


In [23]:
def get_users() -> List[User]:
    qry = '''
        SELECT
            "u"."id",
            CASE
                WHEN "ea"."status" = 'approved' THEN TRUE
                WHEN "ea"."status" = 'rejected' THEN FALSE
                ELSE NULL
            END AS "bs_success",
            "ea"."rejectReasonCode" AS "bs_fail_reason",
            EXISTS (
                SELECT 1
                FROM "loan_assessment"
                WHERE "userId" = "u"."id" AND "cbStatus" IS NOT NULL
            ) AS "cb_done",
            CASE
                WHEN EXISTS (
                    SELECT 1
                    FROM "loan_assessment"
                    WHERE "userId" = "u"."id" AND "cbStatus" = 'rejected'
                ) THEN FALSE
                ELSE TRUE
            END AS "cb_success"
        FROM "user" "u"
        LEFT JOIN LATERAL (
            SELECT *
            FROM "earning_assessment"
            WHERE
                "userId" = "u"."id"
                AND "status" != 'calculating'
                AND "userTestId" IS NULL
            ORDER BY "createdAt" DESC
            LIMIT 1
        ) "ea" ON TRUE
        WHERE
            EXISTS (
                SELECT 1
                FROM "bs_parse_request" "r"
                INNER JOIN "bank_account" "a"
                    ON "a"."id" = "r"."bankAccountId"
                WHERE
                    "a"."userId" = "u"."id"
                    AND "r"."status" = 'completed'
            ) AND "ea"."status" IS NOT NULL
        ORDER BY "u"."id" ASC;
    '''
    try:
        with conn.cursor() as cursor:
            cursor.execute(qry)
            res = cursor.fetchall()
    except Exception as e:
        conn.rollback()
        raise e
    return [User(id=row[0], bs_success=row[1], bs_fail_reason=row[2], cb_done=row[3], cb_success=row[4]) for row in res]

users = get_users()
users

[User(id=7, bs_success=False, bs_fail_reason='balance', cb_done=True, cb_success=False),
 User(id=42, bs_success=True, bs_fail_reason=None, cb_done=True, cb_success=False),
 User(id=74, bs_success=True, bs_fail_reason=None, cb_done=True, cb_success=False),
 User(id=87, bs_success=False, bs_fail_reason='spending', cb_done=True, cb_success=False),
 User(id=98, bs_success=False, bs_fail_reason='income', cb_done=True, cb_success=False),
 User(id=101, bs_success=True, bs_fail_reason=None, cb_done=True, cb_success=False),
 User(id=103, bs_success=True, bs_fail_reason=None, cb_done=True, cb_success=False),
 User(id=109, bs_success=False, bs_fail_reason='balance', cb_done=True, cb_success=False),
 User(id=116, bs_success=False, bs_fail_reason='balance', cb_done=True, cb_success=False),
 User(id=127, bs_success=True, bs_fail_reason=None, cb_done=True, cb_success=True),
 User(id=162, bs_success=True, bs_fail_reason=None, cb_done=False, cb_success=True),
 User(id=163, bs_success=True, bs_fail_rea

In [24]:
@dataclass
class UserBsResult:
    has_recent_data: bool # 최근 데이터 (30일 내) 존재 여부
    recent_data_date: Optional[date] # 최근 데이터 날짜
    recent_90_spending_success: bool # 최근 90일 소비 성공 여부
    recent_90_spending_count: int # 최근 90일 소비 개수
    recent_30_spending_success: bool # 최근 30일 소비 성공 여부
    recent_30_spending_count: int # 최근 30일 소비 개수
    less_than_500_days_count: int # 잔고 500나이라 미만 날짜 개수
    more_than_5000_days_count: int # 잔고 5000나이라 이상 날짜 개수
    balance_composite_success: bool # 잔고 복합룰 성공 여부
    balance_maintain_success: bool # 잔고 유지룰 성공 여부
    balance_empty_success: bool # 잔고 0원룰 성공 여부
    balance_recent_maintain_success: bool # 최근 잔고 유지룰 성공 여부
    balance_recent_empty_success: bool # 최근 잔고 0원룰 성공 여부
    week_0_weekday: Optional[date] # 0번째 주 날짜
    week_0_pay: float # 0번째 주 주급
    week_1_weekday: Optional[date] # 1번째 주 날짜
    week_1_pay: float # 1번째 주 주급
    week_2_weekday: Optional[date] # 2번째 주 날짜
    week_2_pay: float # 2번째 주 주급
    week_3_weekday: Optional[date] # 3번째 주 날짜
    week_3_pay: float # 3번째 주 주급
    week_4_weekday: Optional[date] # 4번째 주 날짜
    week_4_pay: float # 4번째 주 주급
    week_5_weekday: Optional[date] # 5번째 주 날짜
    week_5_pay: float # 5번째 주 주급
    week_6_weekday: Optional[date] # 6번째 주 날짜
    week_6_pay: float # 6번째 주 주급
    week_7_weekday: Optional[date] # 7번째 주 날짜
    week_7_pay: float # 7번째 주 주급
    week_8_weekday: Optional[date] # 8번째 주 날짜
    week_8_pay: float # 8번째 주 주급
    week_9_weekday: Optional[date] # 9번째 주 날짜
    week_9_pay: float # 9번째 주 주급
    week_10_weekday: Optional[date] # 10번째 주 날짜
    week_10_pay: float # 10번째 주 주급
    week_11_weekday: Optional[date] # 11번째 주 날짜
    week_11_pay: float # 11번째 주 주급
    weekly_pay_success: bool # 주급 성공 여부
    month_0_pay: float # 0번째 월 월급
    month_1_pay: float # 1번째 월 월급
    month_2_pay: float # 2번째 월 월급
    monthly_pay_success: bool # 월급 성공 여부
    occasional_pay_success: bool # 비정기급 성공 여부
    gambling_average_spend_amount: float # 도박 평균 소비 금액
    gambling_month_average_spend_count: float # 도박 월 평균 소비 횟수
    gambling_rule_1_success: bool # 도박 룰 1 성공 여부
    gambling_month_average_deposit_amount: float # 도박 월 평균 입금 금액
    gambling_month_average_spend_amount: float # 도박 월 평균 소비 금액
    gambling_rule_2_success: bool # 도박 룰 2 성공 여부

    def to_dict(self) -> dict:
        return {k: v for k, v in self.__dict__.items() if v is not None}


In [25]:
@dataclass
class DailyBalance:
    date: date
    balance: float

def calculate_daily_account_balance(
    bank_account_id: int,
    now: datetime,
    *,
    is_opay_account: bool = False,
    wallet_type: Optional[str] = None
) -> List[DailyBalance]:
    """
    최근 90일간의 계좌 잔고를 계산합니다.
    
    Args:
        bank_account_id: 은행 계좌 ID
        now: 현재 시간
        wallet_type: OPay 지갑 타입 (선택사항)
    
    Returns:
        날짜별 잔고 목록
    """
    today = start_of_day(now)
    start_date = today - timedelta(days=89)
    
    # OPay 계좌 처리
    if is_opay_account and not wallet_type:
        wallet_balances = calculate_daily_account_balance(bank_account_id, now, wallet_type='wallet')
        owealth_balances = calculate_daily_account_balance(bank_account_id, now, wallet_type='owealth')
        
        result = []
        current_date = start_date
        while current_date <= today:
            wallet_balance = next((b.balance for b in wallet_balances if b.date == current_date.date()), 0)
            owealth_balance = next((b.balance for b in owealth_balances if b.date == current_date.date()), 0)
            total_balance = wallet_balance + owealth_balance
            
            if total_balance:
                result.append(DailyBalance(date=current_date.date(), balance=total_balance))
            current_date += timedelta(days=1)
        return result

    # 최근 90일간의 트랜잭션 조회
    qry = '''
        SELECT 
            "bt"."transactionAt",
            "bt"."balance",
            "bt"."amount",
            "bt"."type"
        FROM "bank_transaction" "bt"
        WHERE 
            "bt"."bankAccountId" = %s
            AND "bt"."transactionAt" >= %s
            AND "bt"."balance" IS NOT NULL
    '''
    
    params = [bank_account_id, start_date.strftime('%Y-%m-%dT00:00:00%z')]
    
    if wallet_type:
        qry += ' AND "bt"."oPayWalletType" = %s'
        params.append(wallet_type)
    
    qry += ' ORDER BY "bt"."transactionAt" ASC, "bt"."id" ASC'
    
    transactions = run_query(qry, tuple(params))
    
    # 날짜별 마지막 트랜잭션의 잔고를 저장할 딕셔너리
    daily_balance_map = {}
    
    # 각 트랜잭션을 날짜별로 그룹핑하고 마지막 잔고 선택
    for trans_date, balance, _, _ in transactions:
        date_key = trans_date.strftime('%Y-%m-%d')
        if balance is not None:
            daily_balance_map[date_key] = float(balance)
    
    # 시작일의 잔고가 없을 경우 이전 트랜잭션에서 계산
    start_date_key = start_date.strftime('%Y-%m-%d')
    if start_date_key not in daily_balance_map:
        qry = '''
            SELECT 
                "bt"."balance",
                "bt"."amount",
                "bt"."type"
            FROM "bank_transaction" "bt"
            WHERE 
                "bt"."bankAccountId" = %s
                AND "bt"."transactionAt" >= %s
                AND "bt"."balance" IS NOT NULL
            ORDER BY "bt"."transactionAt" ASC
            LIMIT 1
        '''
        prev_transaction = run_query(qry, (bank_account_id, start_date.strftime('%Y-%m-%dT00:00:00%z')))
        
        if prev_transaction:
            balance, amount, trans_type = prev_transaction[0]
            last_balance = float(balance) + float(amount) if trans_type == 'debit' else float(balance) - float(amount)
        else:
            last_balance = 0
    else:
        last_balance = daily_balance_map[start_date_key]
    
    # 90일간의 모든 날짜에 대한 잔고 계산
    result = []
    current_date = start_date
    
    while current_date <= today:
        date_key = current_date.strftime('%Y-%m-%d')
        balance = daily_balance_map.get(date_key)
        
        if balance is None:
            result.append(DailyBalance(date=current_date.date(), balance=last_balance))
        else:
            result.append(DailyBalance(date=current_date.date(), balance=balance))
            last_balance = balance
            
        current_date += timedelta(days=1)
    
    return sorted(result, key=lambda x: x.date)

In [26]:
def get_user_daily_balance(user: User, now: datetime) -> List[DailyBalance]:
    """
    유저의 모든 은행 계좌에 대한 최근 90일간의 잔고를 계산합니다.
    
    Args:
        user: 사용자
        now: 현재 시간
    
    Returns:
        날짜별 총 잔고 목록
    """
    # 사용자의 유효한 은행 계좌 조회
    qry = '''
        SELECT
            "ba"."id",
            CASE
                WHEN "b"."parseFunctionName" = 'opay' THEN TRUE
                ELSE FALSE
            END AS "is_opay_account"
        FROM "bank_account" "ba"
        INNER JOIN "bank" "b"
            ON "b"."id" = "ba"."bankId"
        WHERE "ba"."userId" = %s
            AND "ba"."forCertification" = true
            AND "ba"."discardedAt" IS NULL
    '''
    bank_accounts = run_query(qry, (user.id,))
    
    # 각 계좌별 일일 잔고 계산
    all_balances: List[List[DailyBalance]] = []
    for bank_account_id, is_opay_account in bank_accounts:
        account_balances = calculate_daily_account_balance(bank_account_id, now, is_opay_account=is_opay_account)
        all_balances.append(account_balances)
    
    # 날짜별 총 잔고 계산
    start_date = start_of_day(now - timedelta(days=89))
    end_date = start_of_day(now)
    result = []
    
    current_date = start_date
    while current_date <= end_date:
        # 현재 날짜의 모든 계좌 잔고 합산
        total_balance = 0
        for account_balances in all_balances:
            day_balance = next(
                (b.balance for b in account_balances if b.date == current_date.date()),
                0
            )
            total_balance += day_balance
        
        result.append(DailyBalance(
            date=current_date.date(),
            balance=total_balance
        ))
        current_date += timedelta(days=1)
    
    return result

In [27]:
def get_user_bs_result(user: User) -> UserBsResult:
    qry = '''
        SELECT "bt"."transactionAt"
        FROM "bank_transaction" "bt"
        INNER JOIN "bank_account" "ba"
            ON "ba"."id" = "bt"."bankAccountId"
        WHERE
            "ba"."userId" = %s
            AND "bt"."transactionAt" >= %s
    '''
    _30_days_ago = start_of_day(now - timedelta(days=30))
    res = run_query(qry, (user.id, _30_days_ago.strftime('%Y-%m-%dT00:00:00%z')))
    if len(res) == 0:
        has_recent_data = False
        recent_data_date = None
    else:
        has_recent_data = True
        recent_data_date = res[0][0].strftime('%Y-%m-%d')

    qry = '''
        SELECT COUNT(*)
        FROM "bank_transaction" "bt"
        INNER JOIN "bank_account" "ba"
            ON "ba"."id" = "bt"."bankAccountId"
        WHERE
            "ba"."userId" = %s
            AND "bt"."transactionAt" >= %s
    '''
    _90_days_ago = start_of_day(now - timedelta(days=90))
    res = run_query(qry, (user.id, _90_days_ago.strftime('%Y-%m-%dT00:00:00%z')))
    recent_90_spending_count = res[0][0]
    recent_90_spending_success = recent_90_spending_count >= 20
    res = run_query(qry, (user.id, _30_days_ago.strftime('%Y-%m-%dT00:00:00%z')))
    recent_30_spending_count = res[0][0]
    recent_30_spending_success = recent_30_spending_count >= 5

    user_daily_balance = get_user_daily_balance(user, now)

    less_than_500_days = [x for x in user_daily_balance if x.balance < 50000]
    less_than_500_days_count = len(less_than_500_days)
    more_than_5000_days = [x for x in user_daily_balance if x.balance >= 500000]
    more_than_5000_days_count = len(more_than_5000_days)

    balance_composite_success = less_than_500_days_count < 40 or more_than_5000_days_count >= 15
    balance_maintain_success = more_than_5000_days_count >= 10
    balance_empty_success = less_than_500_days_count < 48

    recent_30_balance = user_daily_balance[-30:]
    recent_more_than_5000_days = [x for x in recent_30_balance if x.balance >= 500000]
    recent_more_than_5000_days_count = len(recent_more_than_5000_days)
    balance_recent_maintain_success = recent_more_than_5000_days_count >= 3

    recent_less_than_500_days = [x for x in recent_30_balance if x.balance < 50000]
    recent_less_than_500_days_count = len(recent_less_than_500_days)
    balance_recent_empty_success = recent_less_than_500_days_count < 10

    qry = '''
        SELECT "bt"."transactionAt", "bt"."amount"
        FROM "bank_transaction" "bt"
        INNER JOIN "bank_account" "ba" ON "ba"."id" = "bt"."bankAccountId"
        WHERE "ba"."userId" = %s
            AND "bt"."type" = 'credit'
            AND "bt"."transactionAt" >= %s
            AND NOT EXISTS (
                SELECT 1
                FROM "bank_transaction_category" "btc"
                WHERE "btc"."bankTransactionId" = "bt"."id"
                    AND "btc"."type" IN ('gambling', 'simple', 'opay_autosave')
            )
        ORDER BY "bt"."transactionAt" ASC;
    '''
    transactions = run_query(qry, (user.id, _90_days_ago.strftime('%Y-%m-%dT00:00:00%z')))

    # 주급 체크
    weekly_deposits = {}
    for trans_date, amount in transactions:
        week_key = (trans_date - timedelta(days=trans_date.weekday())).strftime('%Y-%m-%d')
        weekly_deposits[week_key] = weekly_deposits.get(week_key, 0) + amount
    
    weeks_with_sufficient_deposit = sum(1 for amount in weekly_deposits.values() if amount >= 500000)  # 5,000 나이라
    weekly_deposit_items = list(weekly_deposits.items())
    week_0_weekday = weekly_deposit_items[0][0] if len(weekly_deposit_items) >= 1 else None
    week_0_pay = weekly_deposit_items[0][1] if len(weekly_deposit_items) >= 1 else None
    week_1_weekday = weekly_deposit_items[1][0] if len(weekly_deposit_items) >= 2 else None
    week_1_pay = weekly_deposit_items[1][1] if len(weekly_deposit_items) >= 2 else None
    week_2_weekday = weekly_deposit_items[2][0] if len(weekly_deposit_items) >= 3 else None
    week_2_pay = weekly_deposit_items[2][1] if len(weekly_deposit_items) >= 3 else None
    week_3_weekday = weekly_deposit_items[3][0] if len(weekly_deposit_items) >= 4 else None
    week_3_pay = weekly_deposit_items[3][1] if len(weekly_deposit_items) >= 4 else None
    week_4_weekday = weekly_deposit_items[4][0] if len(weekly_deposit_items) >= 5 else None
    week_4_pay = weekly_deposit_items[4][1] if len(weekly_deposit_items) >= 5 else None
    week_5_weekday = weekly_deposit_items[5][0] if len(weekly_deposit_items) >= 6 else None
    week_5_pay = weekly_deposit_items[5][1] if len(weekly_deposit_items) >= 6 else None
    week_6_weekday = weekly_deposit_items[6][0] if len(weekly_deposit_items) >= 7 else None
    week_6_pay = weekly_deposit_items[6][1] if len(weekly_deposit_items) >= 7 else None
    week_7_weekday = weekly_deposit_items[7][0] if len(weekly_deposit_items) >= 8 else None
    week_7_pay = weekly_deposit_items[7][1] if len(weekly_deposit_items) >= 8 else None
    week_8_weekday = weekly_deposit_items[8][0] if len(weekly_deposit_items) >= 9 else None
    week_8_pay = weekly_deposit_items[8][1] if len(weekly_deposit_items) >= 9 else None
    week_9_weekday = weekly_deposit_items[9][0] if len(weekly_deposit_items) >= 10 else None
    week_9_pay = weekly_deposit_items[9][1] if len(weekly_deposit_items) >= 10 else None
    week_10_weekday = weekly_deposit_items[10][0] if len(weekly_deposit_items) >= 11 else None
    week_10_pay = weekly_deposit_items[10][1] if len(weekly_deposit_items) >= 11 else None
    week_11_weekday = weekly_deposit_items[11][0] if len(weekly_deposit_items) >= 12 else None
    week_11_pay = weekly_deposit_items[11][1] if len(weekly_deposit_items) >= 12 else None
    weekly_pay_success = weeks_with_sufficient_deposit >= 12

    # 월별 입금액 계산
    periods = [
        (now - timedelta(days=29), now),
        (now - timedelta(days=59), now - timedelta(days=30)),
        (now - timedelta(days=89), now - timedelta(days=60))
    ]
    
    monthly_deposits = []
    for start_date, end_date in periods:
        period_transactions = [
            amount for trans_date, amount in transactions
            if start_date <= trans_date <= end_date
        ]
        
        max_deposit = max(period_transactions) if period_transactions else 0
        total_deposit = sum(period_transactions)
        monthly_deposits.append({"max_deposit": max_deposit, "total_deposit": total_deposit})
    
    # 월급 룰 체크 (월간 최대 입금액이 20,000 나이라 이상)
    months_with_sufficient_salary = sum(
        1 for month in monthly_deposits 
        if month["max_deposit"] >= 2000000  # 20,000 나이라
    )
    month_0_pay = monthly_deposits[0]["max_deposit"] if len(monthly_deposits) >= 1 else None
    month_1_pay = monthly_deposits[1]["max_deposit"] if len(monthly_deposits) >= 2 else None
    month_2_pay = monthly_deposits[2]["max_deposit"] if len(monthly_deposits) >= 3 else None
    monthly_pay_success = months_with_sufficient_salary >= 3
    months_with_sufficient_total = sum(
        1 for month in monthly_deposits 
        if month["total_deposit"] >= 20000000  # 200,000 나이라
    )
    occasional_pay_success = months_with_sufficient_total >= 3

    today = start_of_day(now)

    # 제일 오래된 transaction을 검색
    qry = '''
        SELECT "bt"."transactionAt"
        FROM "bank_transaction" "bt"
        INNER JOIN "bank_account" "ba" ON "ba"."id" = "bt"."bankAccountId"
        WHERE "ba"."userId" = %s
        ORDER BY "bt"."transactionAt" ASC
        LIMIT 1
    '''
    oldest_transaction = run_query(qry, (user.id,))
    # 제일 오래된 transaction 날짜부터 오늘까지의 날짜를 "전체 일수"로 계산
    oldest_transaction_date = start_of_day(oldest_transaction[0][0]) if oldest_transaction else None
    total_date_count = (today - oldest_transaction_date).days if oldest_transaction_date else 0

    qry = '''
        SELECT 
            "bt"."type",
            "bt"."amount"
        FROM "bank_transaction" "bt"
        INNER JOIN "bank_account" "ba" ON "ba"."id" = "bt"."bankAccountId"
        WHERE "ba"."userId" = %s
            AND "ba"."forCertification" = true
            AND "ba"."discardedAt" IS NULL
            AND EXISTS (
                SELECT 1
                FROM "bank_transaction_category" "btc"
                WHERE "btc"."bankTransactionId" = "bt"."id"
                    AND "btc"."type" = 'gambling'
            );
    '''
    gambling_transactions = run_query(qry, (user.id,))

    # 도박 소비 트랜잭션 필터링
    gambling_spendings = [
        (trans_type, amount) 
        for trans_type, amount in gambling_transactions 
        if trans_type == 'debit'
    ]

    # 1. 도박 평균 소비 금액 + 소비 횟수 룰
    if gambling_spendings:
        total_spend_amount = sum(amount for _, amount in gambling_spendings)
        gambling_average_spend_amount = total_spend_amount / len(gambling_spendings)
        total_spend_count = len(gambling_spendings)
        gambling_month_average_spend_count = (total_spend_count / total_date_count) * 30 if total_date_count > 0 else 0

        gambling_rule_1_success = gambling_average_spend_amount < 500000 or gambling_month_average_spend_count < 10
    else:
        gambling_average_spend_amount = 0
        gambling_month_average_spend_count = 0
        gambling_rule_1_success = True

    # 2. 도박 소비 총 금액 룰
    total_deposit_amount = sum(
        amount 
        for trans_type, amount in gambling_transactions 
        if trans_type == 'credit'
    ) if gambling_transactions else 0
    total_spend_amount = sum(
        amount 
        for trans_type, amount in gambling_transactions 
        if trans_type == 'debit'
    ) if gambling_transactions else 0
    
    gambling_month_average_deposit_amount = (total_deposit_amount / total_date_count) * 30 if total_date_count > 0 else 0
    gambling_month_average_spend_amount = (total_spend_amount / total_date_count) * 30 if total_date_count > 0 else 0
    
    gambling_rule_2_success = gambling_month_average_spend_amount - gambling_month_average_deposit_amount >= 2000000


    return UserBsResult(
        has_recent_data=has_recent_data,
        recent_data_date=recent_data_date,
        recent_90_spending_success=recent_90_spending_success,
        recent_90_spending_count=recent_90_spending_count,
        recent_30_spending_success=recent_30_spending_success,
        recent_30_spending_count=recent_30_spending_count,
        less_than_500_days_count=less_than_500_days_count,
        more_than_5000_days_count=more_than_5000_days_count,
        balance_composite_success=balance_composite_success,
        balance_maintain_success=balance_maintain_success,
        balance_empty_success=balance_empty_success,
        balance_recent_maintain_success=balance_recent_maintain_success,
        balance_recent_empty_success=balance_recent_empty_success,
        week_0_weekday=week_0_weekday,
        week_0_pay=week_0_pay,
        week_1_weekday=week_1_weekday,
        week_1_pay=week_1_pay,
        week_2_weekday=week_2_weekday,
        week_2_pay=week_2_pay,
        week_3_weekday=week_3_weekday,
        week_3_pay=week_3_pay,
        week_4_weekday=week_4_weekday,
        week_4_pay=week_4_pay,
        week_5_weekday=week_5_weekday,
        week_5_pay=week_5_pay,
        week_6_weekday=week_6_weekday,
        week_6_pay=week_6_pay,
        week_7_weekday=week_7_weekday,
        week_7_pay=week_7_pay,
        week_8_weekday=week_8_weekday,
        week_8_pay=week_8_pay,
        week_9_weekday=week_9_weekday,
        week_9_pay=week_9_pay,
        week_10_weekday=week_10_weekday,
        week_10_pay=week_10_pay,
        week_11_weekday=week_11_weekday,
        week_11_pay=week_11_pay,
        weekly_pay_success=weekly_pay_success,
        month_0_pay=month_0_pay,
        month_1_pay=month_1_pay,
        month_2_pay=month_2_pay,
        monthly_pay_success=monthly_pay_success,
        occasional_pay_success=occasional_pay_success,
        gambling_average_spend_amount=gambling_average_spend_amount,
        gambling_month_average_spend_count=gambling_month_average_spend_count,
        gambling_rule_1_success=gambling_rule_1_success,
        gambling_month_average_deposit_amount=gambling_month_average_deposit_amount,
        gambling_month_average_spend_amount=gambling_month_average_spend_amount,
        gambling_rule_2_success=gambling_rule_2_success,
    )

In [28]:
@dataclass
class CbResult:
    total_lost_count: int # 총 부실 횟수
    recent_loan_success: bool # 최근 대출 룰 성공 여부
    recent_loan_count: int # 최근 (1년) 대출 횟수
    lost_institution_success: int # 부실 기관 룰 성공 여부
    lost_institution_count: int # 부실 기관 수
    recent_small_lost_success: bool # 최근 소액 부실 룰 성공 여부
    recent_small_lost_count: int # 최근 소액 부실 횟수
    first_loan_lost_success: bool # 최초 대출 부실 룰 성공 여부
    first_loan_at: date # 최초 대출 일자
    first_loan_amount: int # 최초 대출 금액

    def to_dict(self) -> dict:
        return {k: v for k, v in self.__dict__.items() if v is not None}


In [29]:
def get_cb_result(user: User) -> CbResult:
    qry = '''
        SELECT COUNT(*)
        FROM "credit_history"
        WHERE "userId" = %s AND "status" = 'lost'
    '''
    res = run_query(qry, (user.id,))
    total_lost_count = res[0][0] if res else 0

    qry = '''
        SELECT COUNT(*)
        FROM "credit_history"
        WHERE
            "userId" = %s
            AND "status" = 'performing'
            AND "openedAt" >= %s
            AND "reportedAt" < %s
            AND "dueAt" < %s
    '''
    _1_year_ago = (now - timedelta(days=365)).strftime('%Y-%m-%d')
    now_ymd = now.strftime('%Y-%m-%d')
    res = run_query(qry, (user.id, _1_year_ago, now_ymd, now_ymd))
    recent_loan_count = res[0][0] if res else 0
    recent_loan_success = total_lost_count == 0 or recent_loan_count >= 2

    qry = '''
        SELECT COUNT(*)
        FROM "credit_history"
        WHERE
            "userId" = %s
            AND "status" = 'lost'
            AND NOT ("identifiedInstitution" = 'aella financial solution ltd' AND "bureau" = 'crc')
        GROUP BY "identifiedInstitution"
    '''
    res = run_query(qry, (user.id,))
    lost_institution_count = len(res)
    lost_institution_success = lost_institution_count < 4

    qry = '''
        SELECT 1
        FROM "credit_history"
        WHERE "userId" = %s
            AND "status" = 'lost'
            AND "openedAt" >= %s
            AND "loanAmount" < 10000
            AND NOT ("identifiedInstitution" = 'aella financial solution ltd' AND "bureau" = 'crc')
    '''
    _2_years_ago = (now - timedelta(days=730)).strftime('%Y-%m-%d')
    res = run_query(qry, (user.id, _2_years_ago))
    recent_small_lost_count = len(res)
    recent_small_lost_success = recent_small_lost_count == 0

    qry = '''
        SELECT "openedAt", "loanAmount", "status", "identifiedInstitution", "bureau"
        FROM "credit_history"
        WHERE "userId" = %s
        ORDER BY "openedAt" ASC
        LIMIT 1
    '''
    res = run_query(qry, (user.id,))
    first_loan_at = res[0][0] if res else None
    first_loan_amount = int(res[0][1]) if res else None
    first_loan_status = res[0][2] if res else None
    first_loan_institution = res[0][3] if res else None
    first_loan_bureau = res[0][4] if res else None
    first_loan_lost_success = first_loan_at is None or first_loan_status != 'lost' or first_loan_at < (now - timedelta(days=365)).date() or first_loan_amount > 15000 or (first_loan_institution == 'aella financial solution ltd' and first_loan_bureau == 'crc')

    return CbResult(
        total_lost_count=total_lost_count,
        recent_loan_success=recent_loan_success,
        recent_loan_count=recent_loan_count,
        lost_institution_success=lost_institution_success,
        lost_institution_count=lost_institution_count,
        recent_small_lost_success=recent_small_lost_success,
        recent_small_lost_count=recent_small_lost_count,
        first_loan_lost_success=first_loan_lost_success,
        first_loan_at=first_loan_at,
        first_loan_amount=first_loan_amount,
    )


In [30]:
def get_user_df(users: List[User]) -> pd.DataFrame:
    result_list = []
    for user in tqdm(users):
        result = {}
        result.update(**user.to_dict())
        result.update(**get_user_bs_result(user).to_dict())
        if user.cb_done:
            result.update(**get_cb_result(user).to_dict())
        result_list.append(result)
    return pd.DataFrame(result_list)

#df = get_user_df(users)
#df.to_csv('user_rule_result.csv', index=False)

# 유저별 잔고 데이터 프레임 만들기

In [31]:
# 빈 데이터프레임 생성
all_users_df = pd.DataFrame(columns=['user_id', 'date', 'balance'])

# 모든 사용자에 대해 반복
for user in users:
    # 각 사용자의 daily balance 가져오기
    res = get_user_daily_balance(user, now)
    
    # 현재 사용자의 데이터프레임 생성
    user_df = pd.DataFrame([
        {
            'user_id': user.id,
            'date': balance.date.strftime('%-m/%-d/%y'),
            'balance': round(balance.balance / 100, 2)  # 100으로 나누기 추가
        }
        for balance in res
    ])
    
    # 전체 데이터프레임에 현재 사용자의 데이터 추가
    all_users_df = pd.concat([all_users_df, user_df], ignore_index=True)

# 컬럼 순서 지정
all_users_df = all_users_df[['user_id', 'date', 'balance']]

ValueError: Invalid format string

In [18]:
all_users_df

,user_id,date,balance
0,7,10/27/24,-1192.93
1,7,10/28/24,-1194.93
2,7,10/29/24,-1192.93
3,7,10/30/24,-1192.93
4,7,10/31/24,-1192.93
...,...,...,...
11065,679,1/20/25,48.88
11066,679,1/21/25,48.88
11067,679,1/22/25,48.88
11068,679,1/23/25,48.88


In [19]:
all_users_df.to_csv('all_users_daily_balance.csv')

# 잔고룰 유저 현황 추가

In [16]:
true_df = pd.read_excel('balance_rule_true.xlsx')
false_df = pd.read_excel('balance_rule_false.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'balance_rule_true.xlsx'

In [27]:
user_id_list = list(true_df['user_id'])

bs_info_list = []

qry = """
        SELECT "user"."earningCertStatus"
        FROM "user"
        WHERE "user"."id" = %s
        """

for uesr_id in user_id_list:
    # CB 탈락
    cursor.execute(qry, (uesr_id,))
    results = cursor.fetchall()

    user_status = results[0][0]

    bs_info_list.append(user_status)

true_df['bs_status'] = bs_info_list

true_df.to_excel('true_real_status.xlsx')

In [31]:
user_id_list = list(false_df['user_id'])

bs_info_list = []

qry = """
        SELECT "user"."earningCertStatus"
        FROM "user"
        WHERE "user"."id" = %s
        """

for uesr_id in user_id_list:
    # CB 탈락
    cursor.execute(qry, (uesr_id,))
    results = cursor.fetchall()

    user_status = results[0][0]

    bs_info_list.append(user_status)

false_df['bs_status'] = bs_info_list

false_df.to_excel('false_real_status.xlsx')